###import data

In [1]:
%load_ext autoreload
%autoreload 2
from keras.models import Model, load_model
import U_net
import read_img
import pickle
import Compute_IOU

X_trian,Y_train = read_img.read_train()
X_test,test_id,test_size = read_img.read_test()


Using TensorFlow backend.


Getting train images and masks ... 


100%|██████████| 670/670 [02:40<00:00,  4.18it/s]

Getting test images ... 



100%|██████████| 65/65 [00:01<00:00, 49.53it/s]


TypeError: u_net_model() got an unexpected keyword argument 'base_num_mask'

In [ ]:
### tunning u-net model

In [4]:
#from keras.models import Model, load_model
#import U_net
#import read_img
#import pickle
#import Compute_IOU

base_num_mask = [16,32,64]
batch_size = [8,16,24,32]
activation = ['elu','relu']
IOU_result = {}
model_num = 1

for mask in base_num_mask:
    for batch in batch_size:
        for a in activation:
            ##U-net model
            model,results = U_net.u_net_model(X_trian,Y_train,base_num_mask =mask ,act = a,batch_size=batch)
            ##save weight
            with open('train_historyDict_model_%d' %(model_num), 'wb') as file_pi:
                pickle.dump(results.history, file_pi)
            model.save_weights('model_weights_%d.h5' %(model_num))
            
            preds_train = model.predict(X_trian[:int(X_trian.shape[0]*0.9)], verbose=1)
            preds_val = model.predict(X_trian[int(X_trian.shape[0]*0.9):], verbose=1)
            y_train = Y_train[:int(X_trian.shape[0]*0.9)]
            Y_val = Y_train[int(X_trian.shape[0]*0.9):]
            
            train_score = Compute_IOU.iou_metric(y_train.reshape(603,256,256),preds_train.reshape(603,256,256))
            val_score = Compute_IOU.iou_metric(Y_val.reshape(67,256,256),preds_val.reshape(67,256,256))
            IOU_result[(model_num,mask,batch,a)] = (train_score,val_score)
                
            model_num+=1

for model_num,mask,batch,a in sorted(IOU_result):
    train_score,val_score = IOU_result[(model_num,mask,batch,a)]
    print('%d,num mask :%d,batch size: %d, activation %s, train score: %f, val score: %f '%(model_num,mask,batch,a,train_score,val_score))
    

initializing U Net model.......
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 256, 256, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 256, 256, 16) 448         lambda_2[0][0]                   
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 256, 256, 16) 0           conv2d_20[0][0]                  
_____________________________________________________________________________

KeyboardInterrupt: 

###Saving parameters (dont run)

In [ ]:
import pickle
with open('train_historyDict_model_1', 'wb') as file_pi:
        pickle.dump(results.history, file_pi)

In [ ]:
import matplotlib.pyplot as plt
#model = model.load_model("model name")

# summarize history for accuracy
plt.plot(results.history['dice_coef'])
plt.plot(results.history['val_dice_coef'])
plt.title('model dice_coef')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
preds_train = model.predict(X_trian[:int(X_trian.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_trian[int(X_trian.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)
# Threshold predictions
import numpy as np
from skimage.transform import resize
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]),(test_size[i][0], test_size[i][1]), 
                                       mode='constant', preserve_range=True))
    
import run_length_encoding

rles,new_test_ids = run_length_encoding.to_runlength_enciding(preds_test_upsampled,test_id)


### create submission file
import pandas as pd
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('finalproject_run_01.csv', index=False)

In [ ]:
import Compute_IOU

preds_train = model.predict(X_trian[:int(X_trian.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_trian[int(X_trian.shape[0]*0.9):], verbose=1)
y_train = Y_train[:int(X_trian.shape[0]*0.9)]
Y_val = Y_train[int(X_trian.shape[0]*0.9):]
print(Compute_IOU.iou_metric(y_train.reshape(603,256,256),preds_train.reshape(603,256,256)))
print(Compute_IOU.iou_metric(Y_val.reshape(67,256,256),preds_val.reshape(67,256,256)))